In [6]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightning as L
from torch.utils.data import DataLoader, TensorDataset


In [11]:
class LightningLSTM(L.LightningModule):
    def __init__(self):

        super().__init__()

        self.lstm = nn.LSTM(input_size=1, hidden_size=1)


    def forward(self, input):

        input_trans = input.view(len(input), 1)
        lstm_out, temp = self.lstm(input_trans)

        prediction = lstm_out[-1]
        return prediction
    
    def configure_optimizers(self):
        return Adam(self.parameters(), lr=0.1)

    def training_step(self, batch, batch_idx):
        input_i,label_i = batch
        output_i = self.forward(input_i[0])
        loss = (output_i - label_i[0])**2

        self.log("train_loss", loss)

        if (label_i == 0):
            self.log("out_0", output_i)
        else:
            self.log("out_1", output_i)

        return loss

In [12]:
model = LightningLSTM()

print("Company A: Observer = 0 , Predicted = ", model(torch.tensor([0.,0.5,0.25,1.])).detach())

print("Company B: Observer = 1 , Predicted = ", model(torch.tensor([1,0.5,0.25,1.])).detach())


Company A: Observer = 0 , Predicted =  tensor([-0.2383])
Company B: Observer = 1 , Predicted =  tensor([-0.2474])


In [15]:

inputs = torch.tensor([[0.,0.5,0.25,1.],[1,0.5,0.25,1.]])

labels = torch.tensor([0.,1.])

dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

trainer = L.Trainer(max_epochs=300, log_every_n_steps=2)
trainer.fit(model, train_dataloaders=dataloader)

print("Company A: Observer = 0 , Predicted = ", model(torch.tensor([0.,0.5,0.25,1.])).detach())

print("Company B: Observer = 1 , Predicted = ", model(torch.tensor([1,0.5,0.25,1.])).detach())

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name | Type | Params | Mode 
--------------------------------------
0 | lstm | LSTM | 16     | train
--------------------------------------
16        Trainable params
0         Non-trainable params
16        Total params
0.000     Total estimated model params size (MB)
1         Modules in train mode
0         Modules in eval mode


Epoch 299: 100%|██████████| 2/2 [00:00<00:00, 155.31it/s, v_num=1]

`Trainer.fit` stopped: `max_epochs=300` reached.


Epoch 299: 100%|██████████| 2/2 [00:00<00:00, 110.01it/s, v_num=1]
Company A: Observer = 0 , Predicted =  tensor([5.3777e-05])
Company B: Observer = 1 , Predicted =  tensor([0.9772])
